In [174]:
import torch
print(torch.__version__)
from math import log

2.1.2+cpu


In [258]:
D = 4
H = 8
R0 = 0.1
hidden_size = 250

In [277]:
from collections import OrderedDict

model = torch.nn.Sequential(
    OrderedDict([
        ("linear_1", torch.nn.Linear(D * H, hidden_size)),
        ("activation_1", torch.nn.ReLU()),
        ("linear_2", torch.nn.Linear(hidden_size, D + 1))
    ])
)
log_Z = torch.nn.Parameter(torch.tensor(0.0, requires_grad=True))
log_Z_optimizer = torch.optim.Adam([log_Z], 0.01)

In [278]:
optimizer = torch.optim.SGD(
    model.parameters(),  # Передаём все тензоры, учавствующие в градиентном спуске
    lr=0.003,  # learning rate - величина шага градиентного спуска
)


In [279]:
def RIGHT_staaate(state):
    global H, D
    res = []
    for i in range(0, D):
        for j in range(0, H):
            res.append(float(0) + (state[i] == j))
    return res

In [280]:
def reward(x):
    global R0, D, H
    f1 = True
    f2 = True
    for d in range(D):
        if not (0.25 < abs(x[d] / (H - 1) - 0.5) <= 0.5): f1 = False
        if not (0.3 < abs(x[d] / (H - 1) - 0.5) < 0.4): f2 = False
    return (R0 + 0.5 * f1 + 2 * f2)


In [291]:
def loss_fn():
    global H, D, log_Z
    state = [0] * D
    way = []
    while True:
        pred = model(torch.tensor(RIGHT_staaate(state)))
        p_Fs = pred
        p_B = 0
        for i in range(0, D):
            if state[i] == H - 1:
                p_Fs[i] = -float("inf")
            if state[i] > 0:
                p_B += 1
        m = torch.nn.Softmax(dim=0)
        p_Fs = m(p_Fs)
        ind = torch.distributions.categorical.Categorical(p_Fs).sample()
        p_B = max(p_B, 1)
        way.append([torch.log(p_Fs[ind]), 1 / p_B])
        if ind == D:
            way.append([0, 1])
            break
        state[ind] += 1
    loss = log_Z - log(reward(state))
    for i in range(0, len(way) - 1):
        loss += way[i][0] - log(way[i + 1][1])
    loss = loss ** 2
    return loss, state

In [292]:
def random_WAy():
    global H, D
    state = [0] * D
    while True:
        pred = model(torch.tensor(RIGHT_staaate(state)))
        p_Fs = pred
        for i in range(0, D):
            if state[i] == H - 1:
                p_Fs[i] = -float("inf")
        m = torch.nn.Softmax(dim=0)
        p_Fs = m(p_Fs)
        ind = torch.distributions.categorical.Categorical(p_Fs).sample()
        if ind == D:
            break
        state[ind] += 1
    return state

In [293]:
 random_WAy()

[0, 0, 0, 0]

In [294]:
from itertools import product

rewards = torch.zeros(*[H for i in range(D)])
coord_diap = [range(H) for _ in range(D)]

for coord in product(*coord_diap):
    rewards[tuple(coord)] = reward(torch.tensor(coord))

rewards /= rewards.sum()

def loss2(a, samples=1000):
    counter = torch.zeros(*[H for i in range(D)])
    for i in range(samples):
        counter[tuple(a[i])] += 1

    counter /= counter.sum()

    return (rewards - counter).abs().sum()


In [ ]:
a = []
for i in range(0, 100000):
    optimizer.zero_grad()
    log_Z_optimizer.zero_grad()
    x = loss_fn()
    loss = x[0]
    smpl = x[1]
    a.append(smpl)
    loss.backward()
    log_Z_optimizer.step()
    optimizer.step()
    if i % 100 == 0:
        print(i)
        print(loss2(a[-10000:], len(a[-10000:])))